<a href="https://colab.research.google.com/github/ruchithelamp/music/blob/main/Cleaning_AB_and_combining_with_MB_Discogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cleaning AcousticBrainz Files

In [2]:
#mounting Google Drive to this Google Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
import os
from datetime import datetime

#Filepath to cleaned AcousticBrainz source CSVs
folder_path = '/content/drive/My Drive/music_anthro_M2/acousticbrainz/filteredABfiles_jun20'

#Locate all .csv files
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
print(f"Found {len(csv_files)} CSV files")

#Load files individually with safety checks
dataframes = []
for file_path in csv_files:
    try:
        df = pd.read_csv(file_path, low_memory=False)
        dataframes.append(df)
    except Exception as e:
        print(f"Failed to load {file_path}: {e}")

#Concatenate all chunks into one DataFrame
df3 = pd.concat(dataframes, ignore_index=True)
print(f"Combined shape: {df3.shape}")


Mounted at /content/drive
Found 200 CSV files
Combined shape: (1979641, 140)


###Extract and Filter by Year

In [3]:
#Extract year/month/day from metadata_tags_date
def extract_date_components(value):
    if pd.isna(value) or value is None:
        return None, None, None
    value_str = str(value).strip()
    if value_str.startswith("['") and value_str.endswith("']"):
        date_str = value_str[2:-2]
    else:
        date_str = value_str
    if len(date_str) == 4:
        return int(date_str), None, None
    try:
        parsed = datetime.strptime(date_str[:10], '%Y-%m-%d')
        return parsed.year, parsed.month, parsed.day
    except ValueError:
        return None, None, None

#Add year/month/day columns
df3[['year', 'month', 'day']] = df3['metadata_tags_date'].apply(lambda x: pd.Series(extract_date_components(x)))

#Filter to selected years
df3 = df3[df3['year'].isin([2007, 2008, 2009, 2020, 2025])]

#Strip 'highlevel_' prefix from column names
df3.columns = df3.columns.str.replace(r'^highlevel_', '', regex=True)

/tmp/ipython-input-3-4258109056.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df3[['year', 'month', 'day']] = df3['metadata_tags_date'].apply(lambda x: pd.Series(extract_date_components(x)))
/tmp/ipython-input-3-4258109056.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df3[['year', 'month', 'day']] = df3['metadata_tags_date'].apply(lambda x: pd.Series(extract_date_components(x)))
/tmp/ipython-input-3-4258109056.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

### Dropping unneseccary columns

In [4]:
#Droping empty/irrelevant metadata columns
drop_list = [...]  # (same long list you used)

#Dropping only existing columns from the drop list
df3.drop(columns=[col for col in drop_list if col in df3.columns], inplace=True)

### Clean List-Wrapped Strings

In [5]:
strip_list = [
    'metadata_tags_albumartist',
    'metadata_tags_musicbrainz_albumartistid',
    'metadata_tags_albumartistsort',
    'metadata_tags_artists',
    'metadata_tags_artistsort',
    'metadata_tags_musicbrainz_recordingid',
    'metadata_tags_genre'
]

def clean_strings(value):
    if pd.isna(value) or value is None:
        return None
    value_str = str(value).strip()
    return value_str[2:-2] if value_str.startswith("['") and value_str.endswith("']") else value_str

#Apply cleaning to each column in strip_list
for col in strip_list:
    if col in df3.columns:
        df3[col] = df3[col].apply(clean_strings)

### Binarizing High-Level Labels

In [6]:
binary_map = {
    'not_danceable': 1, 'danceable': 0,
    'female': 1, 'male': 0,
    'not_acoustic': 1, 'acoustic': 0,
    'not_aggressive': 1, 'aggressive': 0,
    'electronic': 1, 'not_electronic': 0,
    'happy': 1, 'not_happy': 0,
    'party': 1, 'not_party': 0,
    'relaxed': 1, 'not_relaxed': 0,
    'sad': 1, 'not_sad': 0,
    'bright': 1, 'dark': 0,
    'atonal': 1, 'tonal': 0,
    'instrumental': 1, 'voice': 0
}

cols_to_binarize = [
    'danceability_value', 'gender_value', 'mood_acoustic_value', 'mood_aggressive_value',
    'mood_electronic_value', 'mood_happy_value', 'mood_party_value', 'mood_relaxed_value',
    'mood_sad_value', 'timbre_value', 'tonal_atonal_value', 'voice_instrumental_value'
]

df3[cols_to_binarize] = df3[cols_to_binarize].replace(binary_map)

/tmp/ipython-input-6-124717477.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df3[cols_to_binarize] = df3[cols_to_binarize].replace(binary_map)


### Saving the Cleaned AcousticBrainz Dataset and Viewing a Sample

In [7]:
output_path = '/content/drive/My Drive/music_anthro_M2/acousticbrainz_cleaned.csv'
df3.to_csv(output_path, index=False)
print(f"Saved cleaned dataset to: {output_path}")
df3.head()

Saved cleaned dataset to: /content/drive/My Drive/music_anthro_M2/acousticbrainz_cleaned.csv


,danceability_probability,danceability_value,gender_probability,gender_value,genre_dortmund_probability,genre_dortmund_value,genre_electronic_probability,genre_electronic_value,genre_rosamerica_probability,genre_rosamerica_value,...,metadata_tags__albumartist_latin,metadata_tags__artist_latin,metadata_tags__titlelatin,metadata_tags_wwwartist,metadata_tags_genretype,metadata_tags_dates,metadata_tags_albumtitle,year,month,day
0,0.801988,1,0.977130,0,0.352109,electronic,0.838531,ambient,0.619502,rhy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.0,6.0,6.0
1,0.814099,1,0.887909,1,0.665586,jazz,0.902833,ambient,0.356311,rhy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.0,6.0,6.0
2,0.958137,1,0.714743,1,0.791446,electronic,0.782461,ambient,0.439104,rhy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.0,6.0,6.0
3,0.895864,1,0.988884,0,0.464204,folkcountry,0.860592,ambient,0.592945,cla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.0,6.0,6.0
4,0.957418,1,0.523313,0,0.471526,folkcountry,0.979618,ambient,0.612958,cla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.0,6.0,6.0


##  Merging with Pre-Fuzzy-Matched MusicBrainz and Discogs

###Lots of Runtime and timeout Issues with normal fuzzy matching, so we need to limit fuzzy matching to year and run the matching in parallel

In [1]:
#Install imports and packages
!pip install rapidfuzz
import pandas as pd
from rapidfuzz import process, fuzz
from multiprocessing import Pool, cpu_count
import ast
import re

#Establishing file paths
fuzzy_path = '/content/drive/My Drive/music_anthro_M2/merged_fuzzy_matches.csv'
acoustic_path = '/content/drive/My Drive/music_anthro_M2/acousticbrainz_cleaned.csv'

#Loading cleaned fuzzy-matched MusicBrainz/Discogs dataset
fuzzy_df = pd.read_csv(fuzzy_path, low_memory=False)

#Loading AcousticBrainz
acoustic_df_full = pd.read_csv(acoustic_path, low_memory=False)

#Extract only needed columns for matching logic
acoustic_match_cols = ['metadata_tags_artists', 'metadata_tags_title', 'year']
acoustic_match_df = acoustic_df_full[acoustic_match_cols].copy()

#Dropping rows with missing join key parts
acoustic_df = acoustic_match_df.dropna(subset=['metadata_tags_artists', 'metadata_tags_title', 'year']).copy()

#Cleaning metadata_tags_title
acoustic_df['metadata_tags_title'] = acoustic_df['metadata_tags_title'].str.replace(r"^\['|'\]$", '', regex=True)

#Cleaning metadata_tags_artists and simplifying to first artist only
def clean_artist(value):
    try:
        parsed = ast.literal_eval(value)
        parsed_str = ', '.join(parsed) if isinstance(parsed, list) else str(parsed)
    except:
        parsed_str = str(value)
    return re.split(r"[,/']", parsed_str)[0].strip().lower()

#Apply artist cleaning
acoustic_df['artist_clean'] = acoustic_df['metadata_tags_artists'].apply(clean_artist)
fuzzy_df['artist_clean'] = fuzzy_df['artist_musicbrainz'].astype(str).apply(clean_artist)

#Cleaning track titles
fuzzy_df['title_clean'] = fuzzy_df['track_title_musicbrainz'].astype(str).str.lower().str.strip()
acoustic_df['title_clean'] = acoustic_df['metadata_tags_title'].astype(str).str.lower().str.strip()

#Standardizing year columns
fuzzy_df['release_year_musicbrainz'] = pd.to_numeric(fuzzy_df['release_year_musicbrainz'], errors='coerce').fillna(0).astype(int)
acoustic_df['year'] = pd.to_numeric(acoustic_df['year'], errors='coerce').fillna(0).astype(int)

#Creating join keys
fuzzy_df['join_key'] = (
    fuzzy_df['artist_clean'] + ' - ' +
    fuzzy_df['title_clean'] + ' - ' +
    fuzzy_df['release_year_musicbrainz'].astype(str).str.strip()
)

acoustic_df['join_key'] = (
    acoustic_df['artist_clean'] + ' - ' +
    acoustic_df['title_clean'] + ' - ' +
    acoustic_df['year'].astype(str).str.strip()
)

#Building a dict of acoustic join keys grouped by year
acoustic_by_year = acoustic_df.groupby('year')['join_key'].apply(list).to_dict()

#Defining fuzzy match function
def fuzzy_match_row(row):
    year = row['release_year_musicbrainz']
    if pd.isna(year) or year not in acoustic_by_year:
        return None, 0
    query = row['join_key']
    choices = acoustic_by_year[year]
    best_match = process.extractOne(query, choices, scorer=fuzz.token_sort_ratio)
    if best_match and best_match[1] >= 85:
        return best_match[0], best_match[1]
    return None, 0

#Running fuzzy matching in parallel
with Pool(cpu_count()) as pool:
    results = pool.map(fuzzy_match_row, [row for _, row in fuzzy_df.iterrows()])

#Assigning results
fuzzy_df['matched_key'], fuzzy_df['match_score'] = zip(*results)

#Filter only matched rows before merging
matched_rows = fuzzy_df[fuzzy_df['matched_key'].notna()].copy()

#Creating join_key in full AcousticBrainz data for merging
acoustic_df_full['metadata_tags_title'] = acoustic_df_full['metadata_tags_title'].astype(str).str.replace(r"^\['|'\]$", '', regex=True)
acoustic_df_full['artist_clean'] = acoustic_df_full['metadata_tags_artists'].apply(clean_artist)
acoustic_df_full['title_clean'] = acoustic_df_full['metadata_tags_title'].astype(str).str.lower().str.strip()
acoustic_df_full['year'] = pd.to_numeric(acoustic_df_full['year'], errors='coerce').fillna(0).astype(int)
acoustic_df_full['join_key'] = (
    acoustic_df_full['artist_clean'] + ' - ' +
    acoustic_df_full['title_clean'] + ' - ' +
    acoustic_df_full['year'].astype(str).str.strip()
)

#Merging matched records with full AcousticBrainz dataset
merged_df = matched_rows.merge(
    acoustic_df_full,
    left_on='matched_key',
    right_on='join_key',
    how='left'
)

#Saving final output
output_path = '/content/drive/My Drive/music_anthro_M2/final_fuzzy_matched_by_title_artist.csv'
merged_df.to_csv(output_path, index=False)
print(f"Merged dataset saved to: {output_path}")

#Printing out the match rate
matched = merged_df['metadata_tags_title'].notna().sum()
print(f"Matched {matched} of {len(fuzzy_df)} rows to AcousticBrainz ({matched / len(fuzzy_df):.2%})")

Merged dataset saved to: /content/drive/My Drive/music_anthro_M2/final_fuzzy_matched_by_title_artist.csv
Matched 153 of 9002 rows to AcousticBrainz (1.70%)


###Checking on the quality of these matches given the relatively low fuzzy match threshold

In [2]:
matched_df = merged_df[merged_df['metadata_tags_title'].notna()]

sample_matches = matched_df[['artist_musicbrainz', 'track_title_musicbrainz', 'release_year_musicbrainz',
                             'metadata_tags_artists', 'metadata_tags_title', 'year', 'match_score']].sample(20, random_state=1)
display(sample_matches)

,artist_musicbrainz,track_title_musicbrainz,release_year_musicbrainz,metadata_tags_artists,metadata_tags_title,year,match_score
59,opeth,burden,2008,Opeth,Burden,2008,100.000000
33,manowar,gods of war,2007,Manowar,Gods of War,2007,100.000000
69,scars on broadway,world long gone,2008,Scars on Broadway,World Long Gone,2008,100.000000
127,the prodigy,take me to the hospital,2009,The Prodigy,Take Me to the Hospital,2009,100.000000
31,amy macdonald,l.a.,2007,Amy Macdonald,L.A.,2007,100.000000
107,the prodigy,warrior’s dance,2009,The Prodigy,Warrior’s Dance,2009,100.000000
97,death cab for cutie,i will possess your heart,2008,Death Cab for Cutie,I Will Possess Your Heart,2008,100.000000
14,jay-z,blue magic,2007,JAY Z,Blue Magic,2007,92.000000
53,death cab for cutie,i will possess your heart,2008,Death Cab for Cutie,I Will Possess Your Heart,2008,100.000000
44,no halo,put your hands on,2008,No Halo,Put Your Hands On,2008,100.000000


In [10]:
##Adding lyrics for the songs
#Loading the lyrics dataset
lyrics_path = '/content/drive/My Drive/music_anthro_M2/Discogs_and_musicbrainz_english_lyrics_only.csv'
lyrics_df = pd.read_csv(lyrics_path, low_memory=False)

#Droping duplicates just in case
lyrics_df = lyrics_df.drop_duplicates(subset=['musicbrainz_id'])

#Merging lyrics onto merged_df using 'musicbrainz_id' as key
merged_with_lyrics = merged_df.merge(
    lyrics_df[['musicbrainz_id', 'lyrics']],
    on='musicbrainz_id',
    how='left'
)

#Saving updated dataset
final_output_path = '/content/drive/My Drive/music_anthro_M2/final_musicbrainz_discogs_acousticbrainz_with_lyrics.csv'
merged_with_lyrics.to_csv(final_output_path, index=False)
print(f"Merged dataset with lyrics saved to: {final_output_path}")

#Checking how many rows have lyrics now
has_lyrics = merged_with_lyrics['lyrics'].notna().sum()
print(f"Lyrics available for {has_lyrics} of {len(merged_with_lyrics)} tracks ({has_lyrics / len(merged_with_lyrics):.2%})")


Merged dataset with lyrics saved to: /content/drive/My Drive/music_anthro_M2/final_musicbrainz_discogs_acousticbrainz_with_lyrics.csv
Lyrics available for 146 of 153 tracks (95.42%)
